In [1]:
import cv2
from imwatermark import WatermarkEncoder
from imwatermark import WatermarkDecoder
import numpy as np 
import os 
import json 
import pywt
from tqdm import tqdm 
from PIL import Image, ImageEnhance
from skimage.util import random_noise
from collections import defaultdict
import hashlib

In [2]:
def brightness_attack(img_path, out_path, multi=False):
    brightness = 2.5
    if os.path.exists(out_path) and not multi:
        return
    
    img = Image.open(img_path)
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(brightness)
    img.save(out_path)


In [3]:
def gaussian_noise_attack(img_path, out_path, multi=False):
        std = 0.1
        
        if os.path.exists(out_path) and not multi:
            return 
        
        image = cv2.imread(img_path)
        image = image / 255.0
        # Add Gaussian noise to the image
        noise_sigma = std  # Vary this to change the amount of noise
        noisy_image = random_noise(image, mode='gaussian', var=noise_sigma ** 2)
        # Clip the values to [0, 1] range after adding the noise
        noisy_image = np.clip(noisy_image, 0, 1)
        noisy_image = np.array(255 * noisy_image, dtype='uint8')
        cv2.imwrite(out_path, noisy_image)


In [4]:
def rotate_attack(img_path, out_path, multi=False):
    degree = 30
    expand = 0
    if os.path.exists(out_path) and not multi:
        return 
    
    img = Image.open(img_path)
    img = img.rotate(degree, expand=expand)
    img = img.resize((512,512))
    img.save(out_path)


In [5]:
def jpeg_attack(img_path, out_path, multi=False):
    quality = 50
    if os.path.exists(out_path) and not multi:
        return 
    
    img = Image.open(img_path)
    img.save(out_path, "JPEG", quality=quality)

In [6]:
wm_imgs_path = '/raid/home/ashhar21137/watermarking/dwtDctSvd/dwtDctSvd_watermarked_images'

In [7]:
wm_imgs = os.listdir(wm_imgs_path)

In [8]:
attacks = ["brightness","gaussian_noise","jpeg","rotation"]


In [9]:
attacked_dir = '/raid/home/ashhar21137/watermarking/dwtDctSvd/attacked'

In [10]:
attack_detection = defaultdict(lambda: defaultdict(dict))

In [11]:
# for img in wm_imgs : 
#     img_path = os.path.join(wm_imgs_path,img) 
    


In [12]:
print("----------------- Performing Attacks ------------------")
# count = 0 
for img in tqdm(wm_imgs):
    # if count > 0 : break 
    # if ".png" in image : 
    img_pth = os.path.join(wm_imgs_path, img)
    
    # print("img_pth : ", img_pth)
    
    img_name, img_ext = os.path.splitext(img)
    
    # print("img_name : ",img_name)
    # print("img_ext : ",img_ext)

    for attack in attacks: 
        output_path = os.path.join(f"{attacked_dir}/{attack}", f"{attack}_{img_name + img_ext}")
        # print("output path : ",output_path)
        
        # Ensure the output directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        if attack == "brightness":
            brightness_attack(img_pth, output_path)  # Pass individual paths

        elif attack == "gaussian_noise":
            gaussian_noise_attack(img_pth, output_path)

        elif attack == "rotation" :
            rotate_attack(img_pth,output_path)

        elif attack == "jpeg" : 
            jpeg_attack(img_pth,output_path)
        

    # count += 1


----------------- Performing Attacks ------------------


100%|██████████| 200/200 [00:00<00:00, 13461.19it/s]


In [13]:
paraphrase_detection = defaultdict(lambda: defaultdict(dict))
attack_detection = defaultdict(lambda: defaultdict(dict))


In [14]:
import json 


In [15]:
attacks_op_parent = '/raid/home/ashhar21137/watermarking/dwtDctSvd/attacked'

In [16]:
wm = 'test'

In [17]:
def get_watermark_probability(image_path, expected_watermark, scale=32):
    try:
        # Read the image
        bgr = cv2.imread(image_path)
        
        # Create the decoder
        decoder = WatermarkDecoder('bytes', scale)
        # Decode the watermark
        decoded_watermark_bytes = decoder.decode(bgr, 'dwtDctSvd')
        decoded_watermark = decoded_watermark_bytes.decode('utf-8')
        
        # Compare the decoded watermark with the expected watermark
        if decoded_watermark:
            correct_chars = sum([1 for i, j in zip(decoded_watermark, expected_watermark) if i == j])
            total_chars = max(len(decoded_watermark), len(expected_watermark))
            confidence_score = correct_chars / total_chars
        else:
            decoded_watermark = ''
            confidence_score = 0
    except Exception as e:
        # Handle any decoding errors
        decoded_watermark = ''
        confidence_score = 0
    
    return decoded_watermark, confidence_score

In [18]:
print("----------------- Generating detection results ------------------")
count = 1 
for attack in attacks : 
    print(f" ----- Attack : {attack} -----")

    attacked_path = os.path.join(attacks_op_parent,attack) 
    attacked_ids = os.listdir(attacked_path)

    for img in tqdm(attacked_ids) :
        print(f"----- Count : {count} -----")
        print(f" *** Image : {img} ***")
        l = img.split('_')

        if attack == "gaussian_noise" : 
            id = f"{l[2]}_{l[3]}"
        else : 
            id = f"{l[1]}_{l[2]}"

        print("Image id : ",id)
        
        image_path = os.path.join(attacked_path,img)
        # print(image_path)

        decoded_watermark, confidence_score = get_watermark_probability(image_path, wm)

        # bgr = cv2.imread(image_path)
        # decoder = WatermarkDecoder('bytes', 32)
        # watermark = decoder.decode(bgr, 'dwtDctSvd')
        # print(watermark.decode('utf-8'))


        print("Detection Probability : ", confidence_score)

        attack_detection[id][f"{attack}_det_prob"] = confidence_score
        attack_detection[id][f"{attack}_decoded"] = decoded_watermark

        count += 1 


        # id_prob = 0 
        # detection = 0 
        
        


----------------- Generating detection results ------------------
 ----- Attack : brightness -----


  0%|          | 0/200 [00:00<?, ?it/s]

----- Count : 1 -----
 *** Image : brightness_Image_426038.jpg ***
Image id :  Image_426038.jpg


  1%|          | 2/200 [00:00<00:29,  6.68it/s]

Detection Probability :  0
----- Count : 2 -----
 *** Image : brightness_Image_167603.jpg ***
Image id :  Image_167603.jpg
Detection Probability :  0
----- Count : 3 -----
 *** Image : brightness_Image_311914.jpg ***
Image id :  Image_311914.jpg


...

Detection Probability :  0
----- Count : 794 -----
 *** Image : rotation_Image_352884.jpg ***
Image id :  Image_352884.jpg
Detection Probability :  0
----- Count : 795 -----
 *** Image : rotation_Image_474882.jpg ***
Image id :  Image_474882.jpg
Detection Probability :  0.0
----- Count : 796 -----
 *** Image : rotation_Image_560459.jpg ***
Image id :  Image_560459.jpg


 98%|█████████▊| 197/200 [00:19<00:00,  9.27it/s]

Detection Probability :  0.0
----- Count : 797 -----
 *** Image : rotation_Image_412535.jpg ***
Image id :  Image_412535.jpg
Detection Probability :  0
----- Count : 798 -----
 *** Image : rotation_Image_267688.jpg ***
Image id :  Image_267688.jpg
Detection Probability :  0.0
----- Count : 799 -----
 *** Image : rotation_Image_212091.jpg ***
Image id :  Image_212091.jpg


100%|██████████| 200/200 [00:20<00:00,  9.93it/s]

Detection Probability :  0.0
----- Count : 800 -----
 *** Image : rotation_Image_348850.jpg ***
Image id :  Image_348850.jpg
Detection Probability :  0.0


In [19]:
print("Preparing results json ")
with open('dwtdctsvd_attack_detection.json','w') as file : 
    json.dump(attack_detection,file,indent=4)


Preparing results json 


In [21]:
# def load_image(image_path):
#     """
#     Load an image from the specified file path using PIL.

#     Args:
#     image_path (str): Path to the image file.

#     Returns:
#     PIL.Image.Image: Loaded image object.
#     """
#     return Image.open(image_path)

Parphrase 

In [27]:
import re 
from diffusers import AutoPipelineForText2Image
from diffusers import AutoPipelineForImage2Image
import os 
os.environ["CUDA_VISIBLE_DEVICES"]='1'

import torch 


In [28]:
if torch.cuda.is_available : 
    device = 'cuda'
else : 
    device = 'cpu'

print(device)

cuda


In [29]:
paraphrase_model_id = 'stabilityai/stable-diffusion-xl-base-1.0'
# model_id = 'stabilityai/stable-diffusion-2-1-base'

pipeline_text2image = AutoPipelineForText2Image.from_pretrained(paraphrase_model_id, torch_dtype=torch.float16, variant="fp16", use_safetensors=True, add_watermarker=False).to(device)
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to(device)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [30]:
img_dir = "/raid/home/ashhar21137/watermarking/dwtDctSvd/dwtDctSvd_watermarked_images"

In [31]:
with open('/raid/home/ashhar21137/watermarking/dwtDctSvd/captions_train2014_new_format.json','r') as file : 
    new = json.load(file)

In [32]:
img_ids = []
img_pths = []
imgs = os.listdir(img_dir)
for i in imgs : 
    if '.png' or '.jpg' in i :
        id = re.split('[_.]',i)[1]
        img_ids.append(id) 
        img_pths.append(os.path.join(img_dir,i))



In [33]:
print("Imgs ids : ", img_ids)
print("Images : ",imgs)


Imgs ids :  ['79481', '96557', '280260', '62279', '170558', '215481', '520208', '95989', '504616', '309322', '239728', '341550', '405613', '441488', '420721', '18885', '399956', '93785', '570246', '262845', '49183', '142088', '394326', '524068', '352884', '199628', '160559', '57870', '398494', '241364', '84427', '393493', '483008', '469605', '348850', '387083', '77375', '122108', '540162', '19380', '44404', '345781', '405207', '153946', '457732', '153674', '472925', '388754', '170850', '384029', '62604', '570608', '317237', '299411', '109424', '581674', '239811', '318189', '278347', '90570', '374114', '505126', '8649', '145179', '66514', '462559', '4428', '210932', '257350', '122688', '377694', '438422', '161919', '354444', '287541', '556801', '186233', '95133', '496939', '71631', '371029', '259446', '55627', '334080', '150867', '470072', '181104', '559527', '209967', '444546', '409513', '165499', '351430', '36633', '147303', '474882', '318574', '222016', '236772', '486320', '551125', 

In [34]:
count = 0 
for i in range(len(img_ids)) : 
    id = re.split('[_.]',imgs[i])[1]
    if(img_ids[i] != id ) : 
        print(img_ids)
        count += 1

print(count)

0


In [35]:
wmis = os.listdir(img_dir) 
print("Wmis : ",wmis)


Wmis :  ['Image_79481.jpg', 'Image_96557.jpg', 'Image_280260.jpg', 'Image_62279.jpg', 'Image_170558.jpg', 'Image_215481.jpg', 'Image_520208.jpg', 'Image_95989.jpg', 'Image_504616.jpg', 'Image_309322.jpg', 'Image_239728.jpg', 'Image_341550.jpg', 'Image_405613.jpg', 'Image_441488.jpg', 'Image_420721.jpg', 'Image_18885.jpg', 'Image_399956.jpg', 'Image_93785.jpg', 'Image_570246.jpg', 'Image_262845.jpg', 'Image_49183.jpg', 'Image_142088.jpg', 'Image_394326.jpg', 'Image_524068.jpg', 'Image_352884.jpg', 'Image_199628.jpg', 'Image_160559.jpg', 'Image_57870.jpg', 'Image_398494.jpg', 'Image_241364.jpg', 'Image_84427.jpg', 'Image_393493.jpg', 'Image_483008.jpg', 'Image_469605.jpg', 'Image_348850.jpg', 'Image_387083.jpg', 'Image_77375.jpg', 'Image_122108.jpg', 'Image_540162.jpg', 'Image_19380.jpg', 'Image_44404.jpg', 'Image_345781.jpg', 'Image_405207.jpg', 'Image_153946.jpg', 'Image_457732.jpg', 'Image_153674.jpg', 'Image_472925.jpg', 'Image_388754.jpg', 'Image_170850.jpg', 'Image_384029.jpg', 'Im

In [40]:
save_dir = "/raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1"

In [41]:
wm = 'test'

In [42]:
paraphrase_detection = defaultdict(lambda: defaultdict(dict))

In [43]:
count = 1
for i in tqdm(range(len(wmis))) :

    # if count > 5 : break 

    print(f"---------- Count = {count} : Visual Paraphrasing for the watermarked version of {img_ids[i]}-------------")
    print()

    # print(f"records_512_st_0.35/{img_ids[0]}.txt")
    print("Image name : ",wmis[i])
    
    id = re.split('[_.]',wmis[i])[1]

    print("Image id :",id)
    # print("Image id : ",img_ids[i])

    paraphrase_detection[id]["name"] = wmis[i]

    captions = new['annotations'][id]


    post_img = os.path.join(img_dir,wmis[i]) # watermarked image path 
    image = Image.open(post_img)
    # init_image = load_image(image)
    init_image = image

    gen_image = pipeline(captions, image=init_image, strength=0.1, guidance_scale=7.5).images
    
    # image
    # make_image_grid([init_image, image], rows=2, cols=2)

    directory =  f'{save_dir}/{id}'
    print(f"Saving generated images at {directory}")
    if not os.path.exists(directory):
        os.makedirs(directory)

    for k in range(len(gen_image)) :
        gen_save_dir = os.path.join(directory,f'{id}_gen_{k}.png')
        gen_image[k].save(gen_save_dir)
        print(f"Generated Image saved to {gen_save_dir}")

    # is_watermarked
    num_images = len(gen_image) # 5 images generated from 5 captions for each input image
    print()
    print("Number of images generated : ",num_images)

    print("********** Watermark detection for generated images ***************")

    avg_with_prompt = 0
    num_detected = 0

    for j in range(num_images):
        post_img = os.path.join(directory, f'{id}_gen_{j}.png')
        print(f"caption : {captions[j]}")

        # Use get_watermark_probability to check watermark presence
        _, confidence_score = get_watermark_probability(post_img, expected_watermark=wm)

        detection_result = 1 if confidence_score >= 0.75 else 0
        avg_with_prompt += confidence_score
        num_detected += detection_result

        print(f'Watermark Presence Prob.: {confidence_score}')
    
    paraphrase_detection[id]["avg_det_prob"] = avg_with_prompt / num_images
    paraphrase_detection[id]["avg_det_rate"] = num_detected / num_images

    # count += 1
    # break
    count = count + 1  


  0%|          | 0/200 [00:00<?, ?it/s]

---------- Count = 1 : Visual Paraphrasing for the watermarked version of 79481-------------

Image name :  Image_79481.jpg
Image id : 79481


  0%|          | 0/5 [00:00<?, ?it/s]

Saving generated images at /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/79481
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/79481/79481_gen_0.png
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/79481/79481_gen_1.png
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/79481/79481_gen_2.png
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/79481/79481_gen_3.png
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/79481/79481_gen_4.png

Number of images generated :  5
********** Watermark detection for generated images ***************
caption : A blue motor cycle parked in front of a house next to trash cans.
Watermark Presence Prob.: 0
caption : A blue motorcycle under an umbrella and being held up with a metal stand.
Watermark Presence Prob.: 0
caption : A shiny blue mo

  0%|          | 1/200 [00:03<12:13,  3.69s/it]

...

Watermark Presence Prob.: 0.0
---------- Count = 200 : Visual Paraphrasing for the watermarked version of 454325-------------

Image name :  Image_454325.jpg
Image id : 454325


  0%|          | 0/5 [00:00<?, ?it/s]

Saving generated images at /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/454325
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/454325/454325_gen_0.png
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/454325/454325_gen_1.png
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/454325/454325_gen_2.png
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/454325/454325_gen_3.png
Generated Image saved to /raid/home/ashhar21137/watermarking/dwtDctSvd/dwt_paraphrase_0.1/454325/454325_gen_4.png

Number of images generated :  5
********** Watermark detection for generated images ***************
caption : A modern restroom is equipped with fashionable sinks and urinals surrounded by architectural subway tile.
Watermark Presence Prob.: 0
caption : a bathroom with urinals sinks and towel dispensers
Watermark Presence Prob.:

100%|██████████| 200/200 [10:40<00:00,  3.20s/it]

Watermark Presence Prob.: 0.0


In [44]:
len(paraphrase_detection)

200

In [45]:
with open("dwtdctsvd_paraphrase.json","w") as file : 
    json.dump(paraphrase_detection,file,indent=4)


Before attacks : 


In [ ]:
og_img = 